In [ ]:
from google.colab import files

uploaded = files.upload()  # Upload your video file here
video_path = list(uploaded.keys())[0]
audio_path = video_path.replace('.mp4', '.wav')


Saving 5min.mp4 to 5min.mp4


In [ ]:
import subprocess
# Convert video to wav audio
subprocess.call(['ffmpeg', '-i', video_path, '-vn', '-acodec', 'pcm_s16le', '-ar', '16000', '-ac', '1', audio_path])


0

**TRANSCRIPTION**

---



In [ ]:
import whisper
import pandas as pd

model = whisper.load_model("base")

result = model.transcribe(audio_path, word_timestamps=True)
segments = result['segments']

data = []
previous_end = 0.0

for seg in segments:
    start = seg['start']
    end = seg['end']
    text = seg['text'].strip()
    pause_duration = start - previous_end if previous_end else 0
    data.append([start, end, text, pause_duration])
    previous_end = end

df = pd.DataFrame(data, columns=['start_time', 'end_time', 'text', 'pause_duration'])
df.head()


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


,start_time,end_time,text,pause_duration
0,1.96,4.80,I'm going to talk to you about some things I'v...,0.00
1,5.26,9.12,"Most from experience, some of my hurt and pass...",0.46
2,9.24,10.84,but all of them I do believe are true.,0.12
3,11.54,13.52,"Life is not easy, it is not.",0.70
4,13.70,14.96,Don't try to make it that way.,0.18


In [ ]:
output_csv = video_path.replace('.mp4', '5min_transcription.csv')
df.to_csv(output_csv, index=False)
print(f"Transcription saved to: {output_csv}")


Transcription saved to: 5min5min_transcription.csv


KEY WORD IDENTIFYING

---



In [ ]:
from keybert import KeyBERT
import pandas as pd

df = pd.read_csv("/content/5min5min_transcription.csv")

kw_model = KeyBERT(model='all-MiniLM-L6-v2')

key_snippets = []

for text in df['text']:
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=1)
    snippet = keywords[0][0] if keywords else ""
    key_snippets.append(snippet)

df['key_snippet'] = key_snippets

df.to_csv("transcription_with_keywords.csv", index=False)
df.head()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

,start_time,end_time,text,pause_duration,key_snippet
0,1.96,4.80,I'm going to talk to you about some things I'v...,0.00,learned journey
1,5.26,9.12,"Most from experience, some of my hurt and pass...",0.46,passing practicing
2,9.24,10.84,but all of them I do believe are true.,0.12,believe true
3,11.54,13.52,"Life is not easy, it is not.",0.70,life easy
4,13.70,14.96,Don't try to make it that way.,0.18,don try


**SUMMARIZED VIDEO**

---



In [ ]:
import pandas as pd
import subprocess
import os

df = pd.read_csv("/content/transcription_with_keywords.csv")

video_path = "/content/5min.mp4"
output_dir = "clips"
os.makedirs(output_dir, exist_ok=True)

concat_list_path = "concat_list.txt"

with open(concat_list_path, "w") as concat_file:
    for idx, row in df.iterrows():
        if pd.notna(row['key_snippet']) and row['key_snippet'].strip() != "":
            start = row['start_time']
            duration = row['end_time'] - row['start_time']
            clip_path = f"{output_dir}/clip_{idx}.mp4"

            # FFmpeg trim command
            subprocess.run([
                "ffmpeg", "-y",
                "-ss", str(start),
                "-i", video_path,
                "-t", str(duration),
                "-c:v", "libx264", "-c:a", "aac",
                clip_path
            ])

            concat_file.write(f"file '{clip_path}'\n")

subprocess.run([
    "ffmpeg", "-y",
    "-f", "concat",
    "-safe", "0",
    "-i", concat_list_path,
    "-c", "copy",
    "summary_video.mp4"
])


CompletedProcess(args=['ffmpeg', '-y', '-f', 'concat', '-safe', '0', '-i', 'concat_list.txt', '-c', 'copy', 'summary_video.mp4'], returncode=0)